In [1]:
print("hello")

hello


In [2]:
from keras_facenet import FaceNet

In [32]:
embedding = FaceNet()

In [4]:
from ultralytics import YOLO

In [5]:
model = YOLO("models\\runs\\detect\\train\\weights\\best.pt")

In [6]:
import cv2

In [7]:
import numpy as np 

In [8]:
import os
import numpy as np
from PIL import Image
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC  # Or another classifier, e.g., LogisticRegression, RandomForest
from sklearn.metrics import accuracy_score, classification_report
import cv2  # For basic image loading if needed
import pickle  # For saving/loading the model
from tqdm import tqdm #For progress bar


In [9]:
TRAIN_DIR = "data\\dataset\\dataset\\dataset_split\\augmented_train"
TEST_DIR = "data\\dataset\\dataset\\dataset_split\\test"

In [10]:
def load_data(data_dir):
    images = []
    labels = []
    person_names = []

    for person_name in os.listdir(data_dir):
        person_dir = os.path.join(data_dir, person_name)
        if not os.path.isdir(person_dir):
            continue

        if person_name not in person_names:
            person_names.append(person_name)

        for image_file in os.listdir(person_dir):
            if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Add more extensions if needed
                image_path = os.path.join(person_dir, image_file)
                try:
                    img = Image.open(image_path).convert('RGB') # Ensure images are RGB
                    images.append(np.array(img))  # Convert to numpy array
                    labels.append(person_name)
                except Exception as e:
                    print(f"Error loading image {image_path}: {e}")

    return images, labels, person_names

In [11]:
X_train, y_train_labels, train_person_names = load_data(TRAIN_DIR)
X_test, y_test_labels, test_person_names = load_data(TEST_DIR)

In [12]:
print(f"Loaded {len(X_train)} training images from {len(set(y_train_labels))} identities.")
print(f"Loaded {len(X_test)} testing images from {len(set(y_test_labels))} identities.")

Loaded 7584 training images from 297 identities.
Loaded 407 testing images from 297 identities.


In [13]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train_labels)

LabelEncoder()

In [14]:
y_train = label_encoder.transform(y_train_labels)
y_test = label_encoder.transform(y_test_labels)

In [25]:
def generate_embeddings(images, facenet_model):
    embeddings = []
    for img in tqdm(images, desc="Generating Embeddings"):
        # FaceNet requires images to be a specific size (e.g., 160x160)
        img_resized = cv2.resize(img, (160, 160))  # Or whatever size FaceNet expects
        img_expanded = np.expand_dims(img_resized, axis=0)  # Add batch dimension
        embedding = facenet_model.embeddings(img_expanded)[0] # Get embedding, take from batch
        embeddings.append(embedding)
    return np.array(embeddings)

In [26]:
X_train_embeddings = generate_embeddings(X_train, embedding)
X_test_embeddings = generate_embeddings(X_test, embedding)

Generating Embeddings:   0%|          | 0/7584 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


Generating Embeddings:   0%|          | 1/7584 [00:06<14:02:43,  6.67s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


Generating Embeddings:   0%|          | 2/7584 [00:06<6:00:48,  2.86s/it] 

...

Generating Embeddings: 100%|█████████▉| 405/407 [01:50<00:00,  4.38it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step


Generating Embeddings: 100%|█████████▉| 406/407 [01:50<00:00,  4.30it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


Generating Embeddings: 100%|██████████| 407/407 [01:50<00:00,  3.68it/s]


In [27]:
print("Embeddings shape for training:", X_train_embeddings.shape)
print("Embeddings shape for testing:", X_test_embeddings.shape)

Embeddings shape for training: (7584, 512)
Embeddings shape for testing: (407, 512)


In [28]:
with open("train_embbed.pkl","wb") as f:
    pickle.dump(X_train_embeddings,f)

In [15]:
with open("train_embbed.pkl","rb") as f:
    X_train_embeddings = pickle.load(f)

In [29]:
with open("test_embbed.pkl","wb") as f:
    pickle.dump(X_test_embeddings,f)

In [16]:
with open("test_embbed.pkl","rb") as f:
    X_test_embeddings = pickle.load(f)

In [17]:
cls_model = SVC(kernel='linear', probability=True)

In [18]:
cls_model.fit(X_train_embeddings, y_train)

SVC(kernel='linear', probability=True)

In [19]:
cls_model.score(X_test_embeddings,y_test)

0.8845208845208845

In [33]:
from sklearn.metrics import classification_report

In [20]:
# from xgboost import XGBClassifier

In [22]:
# xg_model = XGBClassifier(
#         objective='multi:softprob',  # For multi-class classification
#         n_estimators=100,
#         learning_rate=0.1,
#         max_depth=5,
#         subsample=0.8,
#         colsample_bytree=0.8,
#         random_state=42
#     )

In [23]:
# xg_model.fit(X_train_embeddings,y_train)

In [24]:
with open("facerecognition.pkl","wb") as f:
    pickle.dump(cls_model,f)

In [26]:
FACENET_INPUT_SIZE = (160, 160) #Expected Input Size of FaceNet
YOLO_INPUT_SIZE = (416, 416) 

In [28]:
def detect_and_recognize_faces(image_path, yolo_model, facenet_model, classifier, label_encoder, confidence_threshold=0.5):
    """
    Detects faces in an image using YOLO, recognizes them using FaceNet and a classifier,
    and draws bounding boxes with names on the image.
    """
    try:
        # Load the image
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        original_height, original_width = img.shape[:2]

        # Preprocess the image for YOLO (resize, normalize, etc.  Adapt this to *your* YOLO preprocessing)
        resized_img = cv2.resize(img, YOLO_INPUT_SIZE)  # Resize to YOLO input size
        img_for_yolo = resized_img / 255.0  # Normalize (if your YOLO model expects this)
        img_for_yolo = np.expand_dims(img_for_yolo, axis=0)  # Add batch dimension

        # Make predictions with YOLO
        detections = yolo_model.predict(img_for_yolo) #Adapt detections to get bounding boxes from YOLO output.
                                                        #This depends entirely on your YOLO implementation.
        # Process the detections (this is highly dependent on your YOLO output format)
        # The following is a *placeholder*. You'll need to adapt this to *your* YOLO model's output.

        boxes = []
        confidences = []
        # This part is a placeholder. You need to adapt it based on your YOLO model's output structure.
        # Assuming detections is a list of bounding boxes with confidence scores:
        # detections = detections[0] # remove batch dimension if exists and necessary for your model
        for detection in detections:
            # Assuming each detection is [x1, y1, x2, y2, confidence, class_probs...]
            # x1, y1, x2, y2 = detection[:4]  # Coordinates of the bounding box
            # confidence = detection[4]      # Confidence score

            # Check if your YOLO model is detecting and giving classes. If so, filter for faces.

            # Example to only grab predictions which are above a threshold for the confidence
            # if confidence > YOLO_CONFIDENCE_THRESHOLD:
            #    boxes.append([x1, y1, x2, y2])
            #    confidences.append(confidence)
            #Example, end
            #Replace this with your post processing for YOLO, to extract the bounding boxes, confidence and class names
            pass #Replace Pass with the post processing of your Yolo model and format

        # Iterate through the detected faces
        for i in range(len(boxes)):

            box = boxes[i]

            #Rescale bounding box to original image size since Yolo was run on resized image
            x1, y1, x2, y2 = box
            x1 = int(x1 * original_width)
            y1 = int(y1 * original_height)
            x2 = int(x2 * original_width)
            y2 = int(y2 * original_height)

            confidence = confidences[i]

            # Extract the face ROI (Region of Interest)
            face_roi = img[y1:y2, x1:x2]

            # Ensure the face ROI is not empty and has a reasonable size
            if face_roi.size == 0 or face_roi.shape[0] < 10 or face_roi.shape[1] < 10:  #Adjust Min size accordingly
                print("Skipping small or empty face ROI.")
                continue


            # Recognize the face
            predicted_name, probability = predict_identity(face_roi, facenet_model, classifier, label_encoder)

            # Draw bounding box and label on the image
            label = f"{predicted_name} ({confidence:.2f})" # Show confidence (optional)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Convert back to BGR for displaying with OpenCV
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        return img_bgr #Return image with detections

    except Exception as e:
        print(f"Error in detect_and_recognize_faces: {e}")
        return None

In [29]:
def predict_identity(face_image, facenet_model, classifier, label_encoder):
    """
    Predicts the identity of a face given an image, FaceNet model,
    trained classifier, and label encoder.
    """
    try:
        # Resize the face image to the expected size of FaceNet
        face_resized = cv2.resize(face_image, FACENET_INPUT_SIZE)
        face_expanded = np.expand_dims(face_resized, axis=0)

        # Generate the embedding for the face
        embedding = facenet_model.embeddings(face_expanded)[0]

        # Predict using the trained classifier
        prediction = classifier.predict([embedding])[0]
        probability = classifier.predict_proba([embedding])[0][prediction]  #Probability of that class

        # Decode the prediction using the label encoder
        predicted_name = label_encoder.inverse_transform([prediction])[0]

        return predicted_name, probability

    except Exception as e:
        print(f"Error during prediction: {e}")
        return "Unknown", 0.0

In [30]:
EXAMPLE_IMAGE_PATH = "data\\test1.jpg"

In [33]:
try:
    recognized_image = detect_and_recognize_faces(EXAMPLE_IMAGE_PATH, model, embedding, cls_model, label_encoder)

    if recognized_image is not None:
        cv2.imshow("Detected and Recognized Faces", recognized_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

except Exception as e:
    print(f"Error processing image: {e}")


Error in detect_and_recognize_faces: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:3845: error: (-215:Assertion failed) !dsize.empty() in function 'cv::hal::resize'



In [46]:
def detect_faces_yolo(frame, yolo_model):
    results = yolo_model(frame, verbose=False)
    faces = []
    boxes = []
    
    for result in results:
        boxes_xyxy = result.boxes.xyxy.cpu().numpy()
        for box in boxes_xyxy:
            x1, y1, x2, y2 = map(int, box)
            face = frame[y1:y2, x1:x2]
            faces.append(face)
            boxes.append((x1, y1, x2, y2))
    
    return faces, boxes

In [47]:
def get_embedding(face_image, embedder):
    # Resize and preprocess face
    face = cv2.resize(face_image, (160, 160))
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face = np.expand_dims(face, axis=0)
    face = (face - 127.5) / 128.0  # FaceNet preprocessing
    
    # Get embedding
    embedding = embedder.embeddings(face)
    return embedding[0]

In [48]:
def realtime_recognition(yolo_weights,cls_model,label_encoder,embeddings):
    # Initialize models (replace with your actual paths)
    
    yolo_model, svc_model, le, embedder = yolo_weights, cls_model,label_encoder,embeddings
    
    # Open webcam
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Mirror the frame
        frame = cv2.flip(frame, 1)
        
        # Detect faces
        faces, boxes = detect_faces_yolo(frame, yolo_model)
        
        # Recognize each face
        for i, (face, box) in enumerate(zip(faces, boxes)):
            x1, y1, x2, y2 = box
            
            try:
                # Get embedding
                embedding = get_embedding(face, embedder)
                
                # Predict identity
                proba = svc_model.predict_proba([embedding])[0]
                max_prob = np.max(proba)
                pred_id = np.argmax(proba)
                
                # Set threshold (adjust as needed)
                if max_prob > 0.7:  # Confidence threshold
                    identity = le.inverse_transform([pred_id])[0]
                    color = (0, 255, 0)  # Green for recognized
                else:
                    identity = "Unknown"
                    color = (0, 0, 255)  # Red for unknown
                
                # Draw rectangle and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                label = f"{identity} ({max_prob:.2f})"
                cv2.putText(frame, label, (x1, y1-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            except Exception as e:
                print(f"Error processing face: {e}")
                continue
        
        # Display frame
        cv2.imshow('Face Recognition', frame)
        
        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [49]:
realtime_recognition(model,cls_model,label_encoder,embedding)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [40]:
def predict_identity(face_image, facenet_model, classifier, label_encoder):
    """
    Predicts the identity of a face given an image, FaceNet model,
    trained classifier, and label encoder.
    """
    try:
        # Resize the face image to the expected size of FaceNet
        face_resized = cv2.resize(face_image, (160, 160)) # Adjust size if needed
        face_expanded = np.expand_dims(face_resized, axis=0)

        # Generate the embedding for the face
        embedding = facenet_model.embeddings(face_expanded)[0]

        # Predict using the trained classifier
        prediction = classifier.predict([embedding])[0]
        probability = classifier.predict_proba([embedding])[0][prediction]  #Probability of that class


        # Decode the prediction using the label encoder
        predicted_name = label_encoder.inverse_transform([prediction])[0]

        return predicted_name, probability

    except Exception as e:
        print(f"Error during prediction: {e}")
        return "Unknown", 0.0

In [42]:
NEW_FACE_IMAGE_PATH = 'data\\arshid4.jpg'

try:
    new_face_image = cv2.imread(NEW_FACE_IMAGE_PATH)
    new_face_image = cv2.cvtColor(new_face_image, cv2.COLOR_BGR2RGB) #Ensure correct color order
    predicted_name, probability = predict_identity(new_face_image, embedding, cls_model, label_encoder)
    print(f"Predicted identity: {predicted_name} with probability: {probability}")

except Exception as e:
    print(f"Error loading/processing the new image: {e}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Predicted identity: Arshid with probability: 0.07684179743032662


In [50]:
from mtcnn import MTCNN
import cv2

# Initialize MTCNN detector
detector = MTCNN()

In [52]:
# Load image
image = cv2.imread("data\\test2.jpg")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # MTCNN expects RGB

# Detect faces
results = detector.detect_faces(image_rgb)

# Draw bounding boxes
for face in results:
    x, y, w, h = face['box']
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Display result
cv2.imshow("MTCNN Face Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Input
from tensorflow.keras.optimizers import Adam

In [55]:
num_classes = len(np.unique(y_train))

In [64]:
model = Sequential([
    Input((512,)),
    Dense(256, activation='relu'),  # FaceNet embeds 128D
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [65]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 297)            │        38,313 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,537 (791.16 KB)

 Trainable params: 202,537 (791.16 KB)

 Non-trainable params: 0 (0.00 B)

In [66]:
history = model.fit(
    X_train_embeddings,
    y_train,
    validation_data=(X_test_embeddings, y_test),
    epochs=20,
    batch_size=32
)

Epoch 1/20
237/237 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.1720 - loss: 4.8008 - val_accuracy: 0.3440 - val_loss: 3.7822
Epoch 2/20
237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5188 - loss: 2.5106 - val_accuracy: 0.5553 - val_loss: 2.0124
Epoch 3/20
237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7107 - loss: 1.4191 - val_accuracy: 0.7125 - val_loss: 1.1980
Epoch 4/20
237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7843 - loss: 1.0198 - val_accuracy: 0.7445 - val_loss: 0.9606
Epoch 5/20
237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8127 - loss: 0.8886 - val_accuracy: 0.8133 - val_loss: 0.7996
Epoch 6/20
237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8253 - loss: 0.7974 - val_accuracy: 0.8108 - val_loss: 0.7686
Epoch 7/20
237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8456 - loss: 0.6998 - val_accuracy: 0.8182 - val_loss: 0.7499
Epoch 8/20
237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8594 - loss: 0.6226 - val_accuracy: 0.

In [89]:
with open("facerecognitionDL.pkl","wb") as f:
    pickle.dump(model,f)

In [90]:
with open("label_encoder.pkl","wb") as f:
    pickle.dump(label_encoder,f)

In [69]:
embedding = FaceNet()

In [70]:
def predict_face(image_path):
    # Load and preprocess the image (without normalization)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # FaceNet expects RGB
    img = cv2.resize(img, (160, 160))           # Resize to FaceNet input shape
    img = np.expand_dims(img, axis=0)           # Add batch dimension (1, 160, 160, 3)
    
    # Get FaceNet embedding (512D vector)
    embedder = embedding.embeddings(img)        # Shape: (1, 512)
    
    # Predict using the trained model
    pred_prob = model.predict(embedder)[0]     # Model output probabilities
    pred_class = np.argmax(pred_prob)           # Predicted class index
    confidence = np.max(pred_prob) * 100        # Confidence score
    
    # Map class index back to person's name
    name = label_encoder.inverse_transform([pred_class])[0]
    
    return name, confidence

In [82]:
name,conf = predict_face("data//sareena.webp")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 707ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


In [83]:
name

np.str_('Serena_Williams')

In [84]:
conf

np.float32(98.707954)

In [87]:
def recognize_live_face(model, label_encoder,embedder):
        
    # Open webcam
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert frame to RGB (MTCNN expects RGB)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detect faces using MTCNN
        faces = detector.detect_faces(frame_rgb)
        
        for face in faces:
            # Extract face bounding box
            x, y, w, h = face['box']
            
            # Ensure coordinates are within frame bounds
            x, y = max(0, x), max(0, y)
            w, h = min(w, frame.shape[1] - x), min(h, frame.shape[0] - y)
            
            # Extract face ROI
            face_roi = frame_rgb[y:y+h, x:x+w]
            
            # Resize to FaceNet input shape (160x160)
            face_resized = cv2.resize(face_roi, (160, 160))
            
            # Get FaceNet embedding (512D vector)
            embedding = embedder.embeddings(np.expand_dims(face_resized, axis=0))
            
            # Predict using your trained model
            pred_prob = model.predict(embedding)[0]
            pred_class = np.argmax(pred_prob)
            confidence = np.max(pred_prob) * 100
            
            # Map class index to person's name
            name = label_encoder.inverse_transform([pred_class])[0]
            
            # Draw bounding box and label
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, 
                        f"{name} ({confidence:.2f}%)", 
                        (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.9, (0, 255, 0), 2)
        
        # Display output
        cv2.imshow("Face Recognition", frame)
        
        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [88]:
recognize_live_face(model,label_encoder,embedding)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━

In [92]:
import inspect
from mtcnn import MTCNN

print(display((inspect.getsource(MTCNN))))

'class MTCNN:\n    """\n    MTCNN class for detecting faces and landmarks through configurable stages.\n    This structure allows skipping certain stages to optimize performance based on the user\'s needs.\n\n    Args:\n        stages (str or list, optional): Defines the pipeline stages. It can be a string to choose from predefined\n                                        configurations or a list of stage classes or instances.\n                                        Options: "face_detection_only", "face_and_landmarks_detection".\n                                        Default is "face_and_landmarks_detection".\n        device (str, optional): The device where the model will be run. Can be "CPU:0", "GPU:0", "GPU:1", ...\n                                Default is "CPU:0".\n                                \n    """\n\n    def __init__(self, stages="face_and_landmarks_detection", device="CPU:0"):\n        if isinstance(stages, str):\n            if stages not in COMMON_STAGES:\n        

None
